In [5]:
import onnx
import onnx_graphsurgeon as gs
import onnxruntime as ort
import torch
import numpy as np
from rich import print

In [6]:
def layer_wise(file,i1,i2):

    ort_session_1 = ort.InferenceSession(file)
    org_outputs = [x.name for x in ort_session_1.get_outputs()]

    model = onnx.load(file)
    for node in model.graph.node:
        for output in node.output:
            if output not in org_outputs:
                model.graph.output.extend([onnx.ValueInfoProto(name=output)])
    
    ort_session = ort.InferenceSession(model.SerializeToString())

    outputs = [x.name for x in ort_session.get_outputs()]
    inputs = [x.name for x in ort_session.get_inputs()]
   


    ort_outs = ort_session.run(None, input_feed={inputs[0]:i1,inputs[1]:i2})


    from collections import OrderedDict
    ort_outs = OrderedDict(zip(outputs, ort_outs))

    print(f"Input1 Shape :{i1.shape }Input1 Value :{i1}")
    print(f"Input1 Shape :{i2.shape }Input1 Value :{i2}")

    for key in ort_outs.keys():
        print(f"Layer : {key} Shape :{ort_outs[key].shape} Outputs :{ort_outs[key]}")

In [7]:
input_value1=np.random.randint(1,5,size=(2,2))
input_value2=np.random.randint(1,5,size=(2,2))

In [8]:
i1 = gs.Variable("input1", np.int32, (2, 2))
i2 = gs.Variable("input2", np.int32, (2, 2))

matmul_node = gs.Node(op="MatMul",
                      name="MatMul_SumMul",
                      inputs=[i1, i2],
                      outputs=[gs.Variable("output_tensor", np.int32)])


graph = gs.Graph(nodes=[matmul_node], inputs=[i1,i2], outputs=[matmul_node.outputs[0]])

onnx_model = gs.export_onnx(graph)
onnx.save(onnx_model, "matrixmul.onnx")


In [9]:
layer_wise("matrixmul.onnx",input_value1,input_value2)

Input1 Shape :(2, 2)Input1 Value :[[2 2]
 [2 1]]

Input1 Shape :(2, 2)Input1 Value :[[4 1]
 [2 3]]

Layer : output_tensor Shape :(2, 2) Outputs :[[12  8]
 [10  5]]